In [6]:
import json
import pandas as pd

with open('../data/req_makeorg_environnement.json', encoding = "utf8") as json_data:
    data_dict = json.load(json_data)

In [7]:
#dataframe individus et quantité de propositions
userid=[]
for i in range(len(data_dict['results'])):
  userid.append(data_dict['results'][i]['userId'])

len(userid)

data_id=pd.DataFrame(columns=['Id','Nb propositions'])
i=0
for val in userid:
  nb=userid.count(val)
  data_id.loc[i]=[val,nb]
  i=i+1

data_id.drop_duplicates(subset ="Id", keep = 'first', inplace=True)

data_id

,Id,Nb propositions
0,8c88f320-4c78-416c-999a-be615e11d390,1
1,0570573c-7b7c-4d4b-85bb-1bf2ea054072,2
2,6a92a4ba-5fec-4ed5-8c6c-3c576c408a2a,1
3,760aa912-9528-4a89-a609-c89dee0fc3ec,1
4,615def47-494b-44b6-aae2-22038f544fed,2
...,...,...
9492,e21373c3-4377-4d42-96a8-86fa2ada6cd3,1
9493,33190a52-9e44-45c5-84a3-fb8afdf5f5e6,1
9497,906f6ed1-659e-4444-9d0b-9d88677451f3,1
9498,c80cc61a-94e0-4699-bc67-cb80aa817a4a,1


In [8]:
#Nb individus 
nbindividus=len(data_id)
nbindividus

4469

In [9]:
#Nombre de propositions
nbpropositions=len(data_dict['results'])
nbpropositions

9501

In [5]:
#Dataframe id utilisateur, âge et code postal associés
data=pd.DataFrame(columns=['Id','Age','Code Postal'])
for i in range(len(data_dict['results'])):
  data.loc[i]=[data_dict['results'][i]['userId'],data_dict['results'][i]['author']['age'],data_dict['results'][i]['author']['postalCode']]

data.drop_duplicates(subset ="Id", keep = 'first', inplace=True)

data

,Id,Age,Code Postal
0,8c88f320-4c78-416c-999a-be615e11d390,None,None
1,0570573c-7b7c-4d4b-85bb-1bf2ea054072,None,None
2,6a92a4ba-5fec-4ed5-8c6c-3c576c408a2a,None,83200
3,760aa912-9528-4a89-a609-c89dee0fc3ec,None,33610
4,615def47-494b-44b6-aae2-22038f544fed,None,None
...,...,...,...
9492,e21373c3-4377-4d42-96a8-86fa2ada6cd3,None,None
9493,33190a52-9e44-45c5-84a3-fb8afdf5f5e6,None,None
9497,906f6ed1-659e-4444-9d0b-9d88677451f3,None,None
9498,c80cc61a-94e0-4699-bc67-cb80aa817a4a,None,None


### Age

In [10]:
age=[]
nb_NA=0
for val in data['Age']:
  if val != None :
      age.append(val)
  else :
      nb_NA= nb_NA + 1
print("Valeurs âge manquantes :",nb_NA)
mean=sum(age)/len(age)
print("L'âge moyen est :",mean)
print("La proportion de valeur manquante est :", nb_NA/nbindividus)

Valeurs âge manquantes : 3093
L'âge moyen est : 43.84375
La proportion de valeur manquante est : 0.6921011411948982


In [11]:
#Dataframe age avec effectif
data_age=pd.DataFrame(columns=['Age','Effectif'])
i=0
for val in age:
  nb=age.count(val)
  data_age.loc[i]=[val,nb]
  i=i+1

data_age.drop_duplicates(subset ="Age", keep = 'first', inplace=True)

data_age

,Age,Effectif
0,90,1
1,29,19
2,17,29
3,48,24
4,47,28
...,...,...
361,14,14
440,83,1
829,79,2
1216,86,1


In [12]:
#Dataframe avec catégrories âge
bins = pd.IntervalIndex.from_tuples([(8, 24), (25, 34), (35, 44),(45,54),(55,64),(65,100)])
data_age['Categories age']=pd.cut(data_age['Age'], bins)

#df_cat_age=data_age
del data_age['Age']

data_age = data_age.groupby('Categories age').sum()

data_age=data_age.reset_index()
data_age['Taux']=data_age['Effectif']/sum(data_age['Effectif'])*100
data_age

,Categories age,Effectif,Taux
0,"(8, 24]",215,16.705517
1,"(25, 34]",213,16.550117
2,"(35, 44]",223,17.327117
3,"(45, 54]",257,19.968920
4,"(55, 64]",248,19.269619
5,"(65, 100]",131,10.178710


In [13]:
#Dataframe Age population générale
#Source : https://www.insee.fr/fr/statistiques/2381474

age_pop=pd.DataFrame({'Categories age': ['(8, 24)', '(25, 34)', '(35, 44)','(45,54)','(55,64)','(65,100)'],'Population':[14091876,7732740,8438121, 8782237,8591286,14250597],'Taux':''})
age_pop['Taux']=age_pop['Population']/61886857*100
age_pop

,Categories age,Population,Taux
0,"(8, 24)",14091876,22.770386
1,"(25, 34)",7732740,12.494963
2,"(35, 44)",8438121,13.634754
3,"(45,54)",8782237,14.190795
4,"(55,64)",8591286,13.882246
5,"(65,100)",14250597,23.026855


In [14]:
#Test d'égalité des proportions
from scipy.stats import ks_2samp
#soit H0: les deux proportions suivent la même loi, même fonctions de répartitions
ks_2samp(age_pop['Taux'], data_age['Taux'])
#pvalue=0.47 donc on accepte l'hypothèse nulle

KstestResult(statistic=0.5, pvalue=0.474025974025974)

In [15]:
#Test indépendance entre deux variables quantitatives / Test de corrélation Pearson
from scipy.stats import pearsonr
#H0 : Variables indépendantes si p-value > 5%
#H1 : Variables non indépendantes si p-value < 5%
pearsonr(age_pop['Taux'], data_age['Taux'])
#pvalue=0.13 donc variables indépendantes

(-0.6870874653166132, 0.1315520827859518)

### Code Postal

In [16]:
cp = []
nb_NA_cp=0
for val in data['Code Postal']:
    if val != None and val !='' :
      cp.append(val)
    else :
      nb_NA_cp= nb_NA_cp + 1
print("Valeurs code postal manquantes :",nb_NA_cp)
print("La proportion de valeur manquante est :", nb_NA_cp/nbindividus)

Valeurs code postal manquantes : 3242
La proportion de valeur manquante est : 0.7254419333184158


In [17]:
data_cp=pd.DataFrame(columns=['Code Postal','Quantité'])
i=0
for val in cp:
  nb=cp.count(val)
  data_cp.loc[i]=[val,nb]
  i=i+1

#on supprime les doublons
data_cp.drop_duplicates(subset ="Code Postal", keep = 'first', inplace=True)

data_cp

,Code Postal,Quantité
0,83200,2
1,33610,1
2,61120,1
3,97410,2
4,44000,14
...,...,...
1221,35420,1
1222,78510,1
1223,91230,1
1224,17110,1


In [18]:
#Regroupement par départements
df_depart=data_cp.assign(Departement=data_cp['Code Postal'])
df_depart=df_depart.reset_index()

del df_depart['Code Postal']
del df_depart['index']

for i in range(len(df_depart)):
  df_depart['Departement'].loc[i]=df_depart['Departement'].loc[i][:2]

df_depart = df_depart.groupby('Departement').sum()
df_depart=df_depart.reset_index()
df_depart

/tmp/ipykernel_37073/1347224545.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_depart['Departement'].loc[i]=df_depart['Departement'].loc[i][:2]


,Departement,Quantité
0,01,5
1,02,5
2,03,5
3,04,2
4,05,1
...,...,...
88,92,53
89,93,17
90,94,24
91,95,16


In [19]:
#Regroupement par régions
reg=pd.read_csv('../data/departements-region.csv')

df_region=df_depart.assign(Region=df_depart['Departement'])
del df_region['Departement']

for i in range(len(df_region)):
  for j in range(len(reg)):
    if df_region['Region'].loc[i]==reg['num_dep'].loc[j]:
      df_region['Region'].loc[i]=reg['region_name'].loc[j]

df_region = df_region.groupby('Region').sum()
df_region=df_region.reset_index()
df_region['Region'].loc[0]='Corse'
df_region['Region'].loc[13]='Ile-de-France'
df_region['Region'].loc[1]='Z'
df_region['Taux']=df_region['Quantité']/sum(df_region['Quantité'])*100
df_region=df_region.sort_values(by=['Region'])

df_region

/tmp/ipykernel_37073/2296697802.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_region['Region'].loc[i]=reg['region_name'].loc[j]
/tmp/ipykernel_37073/2296697802.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_region['Region'].loc[0]='Corse'
/tmp/ipykernel_37073/2296697802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_region['Region'].loc[13]='Ile-de-France'
/tmp/ipykernel_37073/2296697802.py:16: SettingWithCo

,Region,Quantité,Taux
2,Auvergne-Rhône-Alpes,141,11.491443
3,Bourgogne-Franche-Comté,45,3.667482
4,Bretagne,62,5.052975
5,Centre-Val de Loire,40,3.259984
0,Corse,5,0.407498
6,Grand Est,66,5.378973
7,Hauts-de-France,74,6.030970
13,Ile-de-France,311,25.346373
8,Normandie,42,3.422983
9,Nouvelle-Aquitaine,107,8.720456


In [20]:
#Dataframe avec la population par region
#Source : https://www.insee.fr/fr/statistiques/2012713#tableau-TCRD_004_tab1_regions2016
population_region=pd.DataFrame({'Region':['Auvergne-Rhône-Alpes', 'Bourgogne- Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 'Hauts-de-France', 'Ile-de-France', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur"],'Population':[8090442,2784858,3371158, 2561451,349269,5522476,5975757,12324261,3305218,6039092,5985697,3837166,5088998],'Taux':''})
population_region['Taux']=population_region['Population']/65235843*100
population_region

,Region,Population,Taux
0,Auvergne-Rhône-Alpes,8090442,12.401836
1,Bourgogne- Franche-Comté,2784858,4.268908
2,Bretagne,3371158,5.167647
3,Centre-Val de Loire,2561451,3.926447
4,Corse,349269,0.535394
5,Grand Est,5522476,8.465401
6,Hauts-de-France,5975757,9.160236
7,Ile-de-France,12324261,18.891855
8,Normandie,3305218,5.066567
9,Nouvelle-Aquitaine,6039092,9.257322


### Proposition et vote

In [21]:
#Nb Agree moyen par proposition
agree=[]
for i in range(len(data_dict['results'])):
  agree.append(data_dict['results'][i]['votes'][0]['count'])
mean_agree=sum(agree)/len(agree)
print("Nb agree moyen :",mean_agree)

#Proportion moyenne des personnes qui répondent à nouveau
prop=[]
for i in range(len(data_dict['results'])):
  if data_dict['results'][i]['votes'][0]['count'] !=0:
    s=data_dict['results'][i]['votes'][0]['qualifications'][0]['count']+data_dict['results'][i]['votes'][0]['qualifications'][1]['count']+data_dict['results'][i]['votes'][0]['qualifications'][2]['count']
    prop.append(s/data_dict['results'][i]['votes'][0]['count'])
mean_prop=sum(prop)/len(prop)
print("Proportion moyenne 2ème vote :", mean_prop)

#Nb Coup de coeur
cdc=[]
for i in range(len(data_dict['results'])):
  cdc.append(data_dict['results'][i]['votes'][0]['qualifications'][0]['count'])  
mean_cdc=sum(cdc)/len(cdc)
print("Nb coup de coeur moyen :",mean_cdc)

#Nb Réalisable
real=[]
for i in range(len(data_dict['results'])):
  real.append(data_dict['results'][i]['votes'][0]['qualifications'][1]['count'])  
mean_real=sum(real)/len(real)
print("Nb réalisable moyen :",mean_real)

#Nb Banalité
banal=[]
for i in range(len(data_dict['results'])):
  banal.append(data_dict['results'][i]['votes'][0]['qualifications'][2]['count'])  
mean_banal=sum(banal)/len(banal)
print("Nb banalité moyen :",mean_banal)

Nb agree moyen : 109.78244395326807
Proportion moyenne 2ème vote : 0.55760574567116
Nb coup de coeur moyen : 21.811177770760974
Nb réalisable moyen : 33.22671297758131
Nb banalité moyen : 7.720555730975687


In [22]:
#Disagree
disagree=[]
for i in range(len(data_dict['results'])):
  disagree.append(data_dict['results'][i]['votes'][1]['count'])
mean_disagree=sum(disagree)/len(disagree)
print("Nb disagree moyen :",mean_disagree)

#Proportion moyenne des personnes qui répondent à nouveau
prop=[]
for i in range(len(data_dict['results'])):
  if data_dict['results'][i]['votes'][1]['count'] !=0:
    s=data_dict['results'][i]['votes'][1]['qualifications'][0]['count']+data_dict['results'][i]['votes'][1]['qualifications'][1]['count']+data_dict['results'][i]['votes'][1]['qualifications'][2]['count']
    prop.append(s/data_dict['results'][i]['votes'][1]['count'])
mean_prop=sum(prop)/len(prop)
print("Proportion moyenne 2ème vote :", mean_prop)

#Nb Surtout pas
sp=[]
for i in range(len(data_dict['results'])):
  sp.append(data_dict['results'][i]['votes'][1]['qualifications'][0]['count'])  
mean_sp=sum(sp)/len(sp)
print("Nb Surtout Pas moyen :",mean_sp)

#Nb Infaisable
imp=[]
for i in range(len(data_dict['results'])):
  imp.append(data_dict['results'][i]['votes'][1]['qualifications'][1]['count'])  
mean_imp=sum(imp)/len(imp)
print("Nb infaisable moyen :",mean_imp)

#Nb Banalité
banal2=[]
for i in range(len(data_dict['results'])):
  banal2.append(data_dict['results'][i]['votes'][1]['qualifications'][2]['count'])  
mean_banal2=sum(banal2)/len(banal2)
print("Nb banalité moyen :",mean_banal2)

Nb disagree moyen : 14.51341964003789
Proportion moyenne 2ème vote : 0.7177726489440492
Nb Surtout Pas moyen : 3.9901063045995158
Nb infaisable moyen : 3.8872750236817177
Nb banalité moyen : 2.5574150089464265


In [23]:
#Neutre
neutre=[]
for i in range(len(data_dict['results'])):
  neutre.append(data_dict['results'][i]['votes'][2]['count'])
mean_neutre=sum(neutre)/len(neutre)
print("Nb neutre moyen :",mean_neutre)

#Proportion moyenne des personnes qui répondent à nouveau
prop=[]
for i in range(len(data_dict['results'])):
  if data_dict['results'][i]['votes'][2]['count'] !=0:
    s=data_dict['results'][i]['votes'][2]['qualifications'][0]['count']+data_dict['results'][i]['votes'][2]['qualifications'][1]['count']+data_dict['results'][i]['votes'][2]['qualifications'][2]['count']
    prop.append(s/data_dict['results'][i]['votes'][2]['count'])
mean_prop=sum(prop)/len(prop)
print("Proportion moyenne 2ème vote :", mean_prop)

#Nb Pas compris
pc=[]
for i in range(len(data_dict['results'])):
  pc.append(data_dict['results'][i]['votes'][2]['qualifications'][0]['count'])  
mean_pc=sum(pc)/len(pc)
print("Nb Pas Compris moyen :",mean_pc)

#Nb Pas d'avis
pa=[]
for i in range(len(data_dict['results'])):
  pa.append(data_dict['results'][i]['votes'][2]['qualifications'][1]['count'])  
mean_pa=sum(pa)/len(pa)
print("Nb Pas d'Avis moyen :",mean_pa)

#Nb Indifférent
ind=[]
for i in range(len(data_dict['results'])):
  ind.append(data_dict['results'][i]['votes'][2]['qualifications'][2]['count'])  
mean_ind=sum(ind)/len(ind)
print("Nb Indifférent moyen :",mean_ind)

Nb neutre moyen : 18.59962109251658
Proportion moyenne 2ème vote : 0.5375517615658322
Nb Pas Compris moyen : 2.337438164403747
Nb Pas d'Avis moyen : 5.295442584991053
Nb Indifférent moyen : 2.508683296495106


In [24]:
#Taux de réaction sur l'ensemble des propositions :
taux_reac=mean_agree+mean_disagree+mean_neutre
print("Nb de réactions moyennes sur l'ensemble des propositions :", taux_reac)

Nb de réactions moyennes sur l'ensemble des propositions : 142.89548468582254


### Graphiques

In [25]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [44]:
#Age
#Comparaison participant et population générale

fig = go.Figure(data=[
    go.Bar(name='Participants', x=age_pop['Categories age'], y=data_age['Taux'], text=data_age['Taux']),
    go.Bar(name='Population générale', x=age_pop['Categories age'], y=age_pop['Taux'], text=age_pop['Taux'])
])

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(barmode='group', 
                  yaxis=dict(title='Proportion (%)',titlefont_size=16,tickfont_size=14,),
                  legend=dict(x=0.83,y=0.99),
                  title=go.layout.Title(
        text="Comparaison proportion participants et population générale par catégories d'âge <br><sup>https://www.insee.fr/fr/statistiques/2381474</sup>",
        xref="paper",
        x=0
    ),legend_y=1.25, title_y=0.9,font_size=18 )

fig.show()

In [51]:
#Code Postal
#Comparaison participant et population générale

fig = go.Figure(data=[
    go.Bar(name='Participants', x=population_region['Region'], y=df_region['Taux'], text=df_region['Taux']),
    go.Bar(name='Population générale', x=population_region['Region'], y=population_region['Taux'], text=population_region['Taux'])
])

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(barmode='group',
                  yaxis=dict(title='Proportion (%)',titlefont_size=16,tickfont_size=14,),
                  #legend=dict(x=0.005,y=0.99),
                  title=go.layout.Title(
        text="Comparaison proportion participants et population générale par régions <br><sup>https://www.insee.fr/fr/statistiques/2012713#tableau-TCRD_004_tab1_regions2016</sup>",
        xref="paper",
        x=0
    ),legend_y=1.25, legend_x=0.8, title_y=0.9,font_size=15
                  )
fig.show()

In [52]:
#Votes au 1er tour

x=['Agree', 'Disagree', 'Neutral']
y=[mean_agree, mean_disagree, mean_neutre]
fig = go.Figure([go.Bar(x=x, y=y)])
fig.update_layout(title_text="Nombre de votes moyen par proposition", title_y=0.9,font_size=18 )
fig.show()

In [56]:
#Votes au 2ème tour

x1=['Coup de coeur','Réalisable','Banalité (+)']
y1=[mean_cdc, mean_real,mean_banal]

x2=['Surtout pas','Infaisable','Banalité (-)']
y2=[mean_sp, mean_imp,mean_banal2]

x3=['Pas compris',"Pas d'avis",'Indifférent']
y3=[mean_pc, mean_pa,mean_ind]

fig = go.Figure(data=[
    go.Bar(name='Agree', x=x1, y=y1),
    go.Bar(name='Disagree', x=x2, y=y2),
    go.Bar(name='Neutral', x=x3, y=y3)
])

fig.update_layout(barmode='stack')
fig.update_layout(title_text='Nombre de votes moyen par proposition', legend_x=0.9,title_y=0.9,font_size=18 )
fig.show()